# Neural Network Testing: Student Academic Performance

### Objective:

Test the neural network model in neuralnetwork.py on the student-por.csv dataset.

## Background

The NeuralNetwork class did very well in classifying hand-written digits in digit_classification.ipynb. Unfortunately, the dataset I used for digit classification in https://github.com/mgta2/machine-learning-course/blob/main/8_ensemble_methods.ipynb is different from that used in this project. Even the pictures have a different number of pixels (8x8 vs 28x28). Therefore, I can't compare results and can't make any conclusions about using neural nets for digit classification vs decision trees.

Instead of re-running the same experiment on a different dataset, I will follow up on my project in https://github.com/mgta2/machine-learning-capstone-project

This project involved trying to predict a high-school student's academic performance from data collected in school reports and questionnaires. I applied linear regression to the problem. In short, the linear regression model wasn't great at predicting the student's actual mark, but it did grant insight into which factors are considered the most important in influencing their grade (e.g. weekend alcohol consumption vs hours spent studying vs number of class absences). The model also did a pretty good job of predicting which students would pass or fail.

### Warning:

The rest of this document assumes the reader has already read https://github.com/mgta2/machine-learning-capstone-project and so will not repeat information (there's nothing complicated in that project, I just won't re-explain decisions made about handling the data).

In [1]:
import numpy
from neuralnetwork import NeuralNetwork

In [2]:
with open("student-por.csv", "r") as f:
    data = f.readlines()

In [3]:
print(data[0])

school;sex;age;address;famsize;Pstatus;Medu;Fedu;Mjob;Fjob;reason;guardian;traveltime;studytime;failures;schoolsup;famsup;paid;activities;nursery;higher;internet;romantic;famrel;freetime;goout;Dalc;Walc;health;absences;G1;G2;G3



## Transforming the Data

In [4]:
# this code is a little horrendous but I haven't built the NeuralNetwork class to work nicely with pandas.
# this code achieves the same effect as in the capstone project, but with modifications for this neural net.
bad_indices = [0,8,9,10,11,30,31]
good_indices = [15,16,17,18,19,20,21,22]
def transform(datapoint):
    attributes = datapoint.split(";")
    target = int(attributes[32])
    inputs = []
    for i in range(32):
        if i in bad_indices:
            continue
        elif i == 1:
            if attributes[1] == "M":
                inputs.append(1)
            else:
                inputs.append(0.01)
        elif i == 2:
            # age ranges 15-22
            inputs.append((int(attributes[i])-14)/8)
        elif i == 3:
            if attributes[3] == "U":
                inputs.append(1)
            else:
                inputs.append(0.01)
        elif i == 4:
            if attributes[4] == "LE3":
                inputs.append(1)
            else:
                inputs.append(0.01)
        elif i == 5:
            if attributes[5] == "T":
                inputs.append(1)
            else:
                inputs.append(0.01)
        elif i == 6:
            # Medu ranges 0-4
            inputs.append((int(attributes[i])+1)/5)
        elif i == 7:
            # Fedu ranges 0-4
            inputs.append((int(attributes[i])+1)/5)
        elif i == 29:
            # absences ranges 0 to 93
            inputs.append(0.01 + int(attributes[i])/94)
        elif i in good_indices:
            if attributes[i] == "yes":
                inputs.append(1)
            else:
                inputs.append(0.01)
        else:
            inputs.append(int(attributes[i])/5)
    return inputs, target

As in digit_classification.ipynb, I have rescaled the inputs to avoid 0. The targets must also avoid 0 and not be too large (this is done below).

In [5]:
print(transform(data[1]))

([0.01, 0.5, 0.01, 0.01, 0.01, 1.0, 1.0, 0.4, 0.4, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.8, 0.6, 0.8, 0.2, 0.2, 0.6, 0.0525531914893617], 11)


## Training and Testing the Neural Network

The neural network will have 25 input nodes and 21 output nodes. For an initial test, I'll take 40 hidden nodes in a depth 3 network. There isn't much data (only 649 entries) so too many nodes will likely prevent the model from learning much at all. I will take 454 for training and the rest for testing (to mimic the 70-30 train-test split in my capstone project)

In [6]:
net = NeuralNetwork([25,40,21],0.2)

In [7]:
for datapoint in data[1:455]:
    inputs, target = transform(datapoint)
    targets = [0.01]*21
    targets[target] = 0.99
    net.train(inputs,targets)

In [8]:
train_score = 0
for datapoint in data[1:455]:
    inputs, target = transform(datapoint)
    outputs = net.query(inputs)
    label = numpy.argmax(outputs)
    if label == target:
        train_score += 1
print("The training score was:", train_score)
print("The training accuracy was:", train_score/454)

The training score was: 79
The training accuracy was: 0.17400881057268722


In [9]:
test_score = 0
for datapoint in data[455:]:
    inputs, target = transform(datapoint)
    outputs = net.query(inputs)
    label = numpy.argmax(outputs)
    if label == target:
        test_score += 1
print("The testing score was:", test_score)
print("The testing accuracy was:", test_score/195)

The testing score was: 25
The testing accuracy was: 0.1282051282051282


In [10]:
print(outputs) # this is the output from the last tested query.

[[0.0201591 ]
 [0.01744627]
 [0.01759126]
 [0.0200125 ]
 [0.01808597]
 [0.01918188]
 [0.01871485]
 [0.02396965]
 [0.0428814 ]
 [0.04600652]
 [0.078422  ]
 [0.27925442]
 [0.07453217]
 [0.07106508]
 [0.07465506]
 [0.07392684]
 [0.04879248]
 [0.0845522 ]
 [0.03172962]
 [0.02012628]
 [0.01624351]]


### Comments:

The neural network is terrible at predicting the student's grade from 0 to 20.

This is unsurprising. First, at only 454 datapoints to train on, there is very little data available. Second, one would never expect it to be the case that a model could perfectly predict a student's score using only school reports and questionnaires.

Therefore, I will abandon this goal and instead move onto a much more reasonable task - trying to classify student's into pass (score > 9) and fail (score <= 9).

In [11]:
neural_net = NeuralNetwork([25,40,2], 0.2)

In [12]:
for datapoint in data[1:455]:
    inputs, target = transform(datapoint)
    targets = [0.01]*2
    if target > 9:
        targets[1] = 0.99
    else:
        targets[0] = 0.99
    neural_net.train(inputs,targets)

In [13]:
train_score = 0
for datapoint in data[1:455]:
    inputs, target = transform(datapoint)
    outputs = neural_net.query(inputs)
    label = numpy.argmax(outputs)
    if (label == 1 and target > 9) or (label == 0 and target <= 9):
        train_score += 1
print("The training score was:", train_score)
print("The training accuracy was:", train_score/454)

The training score was: 413
The training accuracy was: 0.9096916299559471


In [14]:
test_score = 0
for datapoint in data[455:]:
    inputs, target = transform(datapoint)
    outputs = neural_net.query(inputs)
    label = numpy.argmax(outputs)
    if (label == 1 and target > 9) or (label == 0 and target <= 9):
        test_score += 1
print("The testing score was:", test_score)
print("The testing accuracy was:", test_score/195)

The testing score was: 136
The testing accuracy was: 0.6974358974358974


### Comments:

This was much better than trying to outright predict the student's score, but still not great. Again, the lack of large amounts of data means that this endeavour was likely to fail from the start. It is also unlikely that even the best model, trained on huge amounts of data, would ever be reliable in predicting a student's pass/fail chance based solely off self-reported questionnaires and school reports.

That said, the linear regression model had a pass/fail prediction accuracy of 84%, while the above neural network only achieved 70%.

## Conclusion

The Neural Network did not perform amazingly here. The linear regression model seems to be able to do more with less data. This intuitively makes sense. Linear regression attempts to find a hyperplane which best fits the data. Linear relationships are simple models and often don't need much data before a pattern emerges. Neural Networks on the other hand are far more mysterious, and it seems intuitive that large amounts of data are needed before one can get clear output signals.

Still, a correct pass/fail prediction rate of 70% shows that the model was able to extract some information from the data. Therefore, the NeuralNetwork class is functioning as hoped.